<a href="https://colab.research.google.com/github/marriamaslova/compling_nlp_hse_course/blob/master/notebooks/gpt/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов или репа). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
%pip install transformers

In [ ]:
from transformers.utils import logging
logging.set_verbosity(40)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

Для дообучения взяла тексты песен группы "Мельница", в них прослеживается стилистическая и тематическая общность

In [ ]:
text = """ Там за третьим перекрестком,
И оттуда строго к югу,
Всадник с золотою саблей
В травы густо сеет звезды.
Слышишь, гроздьями роняет небо
Из прорех зерно стальное,
Горные лихие тропы
Покрывая пеленою.
Дороги сплелись
В тугой клубок влюбленных змей,
И от дыхания вулканов в туманах немеет крыло...
Лукавый, смирись -
Мы все равно тебя сильней,
И у огней небесных стран
Сегодня будет тепло.
Там у третьего причала
Сизый парус, парус белый,
Делят небо от начала
До рассвета рваной раной,
Слышишь? Море омывает шрамы,
Посыпает крупной солью
Струпья цвета бычьей крови,
Словно память древней боли.
Дороги сплелись
В тугой клубок влюбленных змей,
И от дыхания вулканов в туманах немеет крыло...
Лукавый, смирись -
Мы все равно тебя сильней,
И у огней небесных стран
Сегодня будет тепло.
Там у третьего порога,
За широкою ступенью,
Верно шелковые камни,
Бьется надвое дорога, слышишь?
Правый путь ведет на пристань,
Путь окружный – в горы, к югу,
Но на свете нет дороги,
Чтобы нас вела друг к другу!
Дороги сплелись
В тугой клубок влюбленных змей,
И от дыхания вулканов в туманах немеет крыло...
Лукавый, смирись -
Мы все равно тебя сильней,
И у огней небесных стран
Сегодня будет тепло.
Тугой клубок влюбленных змей,
И от дыхания вулканов в туманах немеет крыло...
Лукавый, смирись -
Мы все равно тебя сильней,
И у огней небесных стран
Сегодня будет тепло.

Закат раскинулся крестом поверх долин вершины грез;
Ты травы завязал узлом и вплел в них прядь моих волос.
Ты слал в чужие сны то сумасшедшее видение страны,
Где дни светлы от света звезд.
Господином Горных Дорог назову тебя;
Кто сказал, что холоден снег?
Перевал пройду и порог, перепутие,
Перекрестье каменных рек.
Я ухожу вослед не знавшим, что значит слово страх.
О, не с тобой ли все пропавшие, погибшие в горах,
Что обрели покой там, где пляшут ветры под твоей рукой
на грани ясного утра?
Господином Горных Дорог назову тебя, облака
Кружат стаей перед грозой.
Наша кровь уходит в песок, позабудь ее, и она
Прорастет тугою лозой.
Я хотела остаться с тобой,
Я уже успела посметь.
Пахнет снегом, прозрачная боль -
То ли даль, то ли высь, то ли смерть...
Пусть укроет цепи следов моих иней,
Чтоб никто найти их не мог.
Кто теперь прочтет подо льдом твое имя,
Господина Горных Дорог?

Вертись вертись мое колесо
Тянись тянись шерстяная нить
Отдавай мой гость мне мое кольцо
А не хочешь если совсем возьми
Отдавай мой гость мне мое кольцо
А не хочешь если совсем возьми
Я себя сегодня не узнаю
То ли сон дурной то ли свет не бел
Отдавай мне душу мой гость мою
А не хочешь если бери себе
Отдавай мне душу мой гость мою
А не хочешь если бери себе
Звон стоит в ушах и трудней дышать
И прядется не шерсть только мягкий шелк
И зачем мне право моя душа
Если ей у тебя мой гость хорошо
И зачем мне право моя душа
Если ей у тебя мой гость хорошо

Прощай, и если навсегда, то навсегда прощай
Когда б за край — иди, прощай и помни обо мне!
Как близко край, а там туман
Январь хохочет, вечно пьян
Я заключён, как истукан, в кольце его огней
Забудь о том, о чём не знал, забудь мои слова
Не мной не сказаны слова, и ты о них забудь
А там за краем рыщет тьма
Как никогда близка зима
И тень твоя, мою обняв, уходит снова в путь
За краем вечности, беспечности, конечности пурги
Когда не с нами были сны, когда мы не смыкали глаз
Мы не проснёмся, не вернёмся ни друг к другу, ни к другим
С обратной стороны зеркального стекла
Когда средь угольев утра ты станешь мне чужой
Когда я стану и тебе чужим, моя душа
Держись за воздух ледяной, за воздух острый и стальной
Он между нами стал стеной, осталось лишь дышать
За краем вечности, беспечности, конечности пурги
Когда не с нами были сны, когда мы не смыкали глаз
Мы не проснёмся, не вернёмся ни друг к другу, ни к другим
С обратной стороны зеркального стекла
За краем ясных и ненастных, и напрасных зимних дней
Когда без звука рвётся синь, когда и ночь без сна бела
Мы не вернёмся ни друг к другу, ни к себе
С обратной стороны зеркального стекла
За краем вечности, беспечности, конечности пурги
Когда не с нами были сны, когда мы не смыкали глаз
Мы не проснёмся, не вернёмся ни друг к другу, ни к другим
С обратной стороны зеркального стекла
За краем ясных, и ненастных, и напрасных зимних дней
Когда без звука рвётся синь, когда и ночь без сна бела
Мы не вернёмся ни друг к другу, ни к себе
С обратной стороны зеркального стекла

А если там, под сердцем, лед,
То почему так больно жжет?
Не потому ли, что у льда
Сестра — кипящая вода,
Которой полон небосвод?
Зима приходит за теплом,
В горячих пальцах снежный ком,
И никаким неверным снам
Не замести дороги нам
В ночь под невидимым крылом.
Ничего не останется от нас,
Нам останемся, может быть, только мы,
И крылатое бьется пламя
Между нами,
Как любовь во время зимы.
Кипит гранит, вертится ось,
Ведь так отроду повелось,
Что всем клинкам и кораблям
Дают девичьи имена —
Что ж остается делать нам?
Ничего не останется от нас,
Нам останемся, может быть, только мы,
И крылатое бьется пламя
Между нами,
Как любовь во время зимы.
Вслепую вновь перелистай
Пергамент нам доступных тайн —
Лёд, раскаленный докрасна,
Любовь страшнее, чем война,
Любовь разит верней, чем сталь.
Вернее, потому что сам
Бежишь навстречу всем ветрам,
Пусть будет боль, и вечный бой,
Не атмосферный, не земной,
Но обязательно — с тобой.
Ничего не останется от нас,
Нам останемся, может быть, только мы,
И крылатое бьется пламя
Между нами,
И любовь во время зимы.
Ничего не останется от нас,
Нам останемся в лучшем случае мы,
Хорошо, что уже не страшно,
И пламя пляшет,
Как любовь во время зимы.
Хорошо, что уже не страшно,
И пламя пляшет,
Как любовь во время зимы.
 """

In [ ]:
# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write(text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

Два запроса с beams + top-k

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Дождь идет 
С обратной стороны зеркального стекла,
И от дыхания вулканов в туманах немеет крыло...
Лукавый, смирись -
Мы все равно тебя сильней, и у огней небесных стран
Сегодня будет тепло.
Там за краем ясных зимних дней
Прорастут гроздьями подснежники —
Когда-то ты любил их колючими иглами
За то, что они не сломались под твоей рукой
В ту ночь, когда небо было беззвездным
Над пропастью недалекой звездной вышины
Тугой змеей извиваясь змейкой
Между нами сплелись тысячи невидимых нитей
Отдавая друг другу последнее тепло
Верней, отдавая себе отчет в том, чего стоите сами
Что имеете право на ошибку?
Ничего не останется от нас, если мы останемся к этому непричастными
Пахнет гнилью прошлогодняя листва, потрескивают сухие ветки
Словно память об ушедших из жизни предках
Нам остается лишь смириться с неизбежным и ждать утра
До тех пор, пока не растают последние леденящие кровь иголки
На грани непроглядной темноты
Ты вернешься ко мне совсем другим, мой ненаглядный
Я снова стану для тебя всем, ч

In [ ]:
text = "Лед сердца "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Лед сердца 
Слышишь, как бьется мое сердце?
Забудь о том, что когда-то было дорого,
И у огней небесных стран
Сегодня будет тепло.
Зима близко, и мы едем в объятья вьюги;
Мы не вернёмся ни друг к другу, ни к другим —
Нам дороги назад нет и быть не может!
Пахнет тиной под копытами коней, лед растекается по стёклам зеркал…
Вставай, страна огромная, вставай же!..
Закрой глаза, закрой их, впусти в себя свет звездный, прими мой дар ясный:
Пусть он померкнет перед грозой ненастной, но я буду с тобой, слышишь?..
Я стану для тебя всем, чем ты есть на самом деле, — человеком, который любит тебя, вещью, которую носит на руках, словом, самым дорогим тебе существом на свете. Возьми меня за руку, прижми к себе покрепче — и все будет хорошо, ничего не останется от нас, останемся только мы с тобою...
Отдавай мне то, во что я так сильно тебя обрела, отдавай мою любовь, нежность, ласку, тепло твоих губ, языка, волос, рук, ног, плеч, груди, спины, шеи, живота, ступней, всего того, без чего немыслимо сущ

Top-k (меняла показатели параметров, оставила то, что дает более-менее адекватный результат)

Вывод результата получился не очень читабельным в Гитхабе, поэтому под ячейкой с кодом продублировала текст, разделив его по строкам

In [ ]:
text = "Она не придет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        top_k=150,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Она не придет 
никогда, слышишь? В вечность без сна и рассвета. Глаза закрой скорей — Туда мне будет от ваших слов тепло… Без тени улыбки на губах Несправедливость губ молчит между нами Пройдет боль веков - Поверь: это навсегда... Поцелуй весны в самый темный день Спроси у тишины рассветной Запей ее страстным горным узлом Просыпайся звездный мой крылатый зверь И скажи тихо тихонько себе Спокойней прощай моя любовь Любовь приходит только с тобой Смотри кто там пришел Скажи когда я вернусь Утро вечера мудренее смотри Все равно дорогу обойдем стороной Расстаемся навеки нас связующая нить Никуда мы тебя теперь уже дороги Тебе ответишь молча промолчи спи спокойно Просто усни устал но знай что нам осталось просто по быть рядом Ночь пройдет лунная совсем разберись Где ключи прячь ложи мою постель Теплый плед теплый уют ветер прохлада свет твой ласкает тело Моя душа словно пламя любви Раствори свое счастье Пойми для начала зачем тебе дан дар Судьба подарила ему совершенную женщину Для счастья

Она не придет 
никогда, слышишь? 

В вечность без сна и рассвета. 

Глаза закрой скорей 

— Туда мне будет от ваших слов тепло… 

Без тени улыбки на губах 

Несправедливость губ молчит между нами 

Пройдет боль веков 

- Поверь: это навсегда... 

Поцелуй весны в самый темный день 

Спроси у тишины рассветной 

Запей ее страстным горным узлом 

Просыпайся звездный мой крылатый зверь И скажи тихо тихонько себе Спокойней 

прощай моя любовь 

Любовь приходит только с тобой

Смотри кто там пришел 

Скажи когда я вернусь 

Утро вечера мудренее 

смотри Все равно дорогу обойдем стороной 

Расстаемся навеки 

нас связующая нить 

Тебе ответишь молча промолчи спи спокойно 

Просто усни устал но знай что нам осталось просто побыть рядом 

Ночь пройдет лунная совсем разберись 

Теплый плед теплый уют ветер прохлада свет твой ласкает тело 

Моя душа словно пламя любви 

Раствори свое счастье 

Пойми для начала зачем тебе дан дар 

Судьба подарила ему совершенную женщину 

Для счастья ей нужен лишь ты Сам мужчина любимый женщина 

Будь счастлив дорогой если бы можно было смотреть как бьется твое сердце пусть оно разобьется о камни 

Но все беды твои разбились тогда 

Прости меня любимая дорогая 

Я никогда больше так сильно ни к кому домой 

Больше туда дорога здесь дверь 

Откройте за мной тяжелая стремительная ночь 

Молчите просите впустите хоть немного тепла 

Наполните наши души светом 

Пускай ваш путь постепенно отдаляясь ото всех Нас сотрясает







Добавила показатель температуры (опять же, привожу лучший вариант из тех значений, которые пробовала)

С выводом результата сделала то же, что в предыдущем случае

In [ ]:
text = "Я найду такого, как ты "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=150,
                        max_length=300,
                        repetition_penalty=3.5,
                        no_repeat_ngram_size=2,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1",
  "use_cache": false
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я найду такого, как ты 
С обратной стороны зеркального стекла. То ли ночь в пути близится к закату - то ль свет от звезд совсем близко... Я тебя обниму на прощание и уведу с собой далеко-далеко; там будет тепло моих теплых пальцев пылающих все утро под струями теплого дождя твоего зимнего рассвета.." (первый куплет)...и небо станет розовым алое нитью через край сверкающей воды моей разгоряченной любви!.. О нет! Не со мной сегодня был этот жестокий удар молнии по моему сердцу -- о нем не может быть речи теперь даже мне самой: я больше никогда его себе этим позволить уже просто так негошеньки нэ смогу!! Мерцание искр над самым ухом обжигает кожу холодными иголками онемевших сосен замерзших липовых аллей парка фортепьяно родного края!!! Это единственное для меня главное счастье во всей этой кромешной тьме непроглядной пурги неприкрытых зонтами зимних вьюг ног навсегда перепрошитых жгучими пальцами ветра колючих игл ледяных нитей моего отчаяния неверия людей вообще входящих порой глубоко 

Я найду такого, как ты 

С обратной стороны зеркального стекла. 

То ли ночь в пути близится к закату - то ль свет от звезд совсем близко... 

Я тебя обниму на прощание и уведу с собой далеко-далеко; там будет тепло моих теплых пальцев пылающих все утро под струями теплого дождя твоего зимнего рассвета.." 

и небо станет розовым алое нитью через край сверкающей воды моей разгоряченной любви!.. 

О нет! Не со мной сегодня был этот жестокий удар молнии по моему сердцу 

-- о нем не может быть речи теперь даже мне самой: я больше никогда его себе этим позволить уже просто так негошеньки нэ смогу!! 

Мерцание искр над самым ухом обжигает кожу холодными иголками онемевших сосен замерзших липовых аллей парка фортепьяно родного края!!! 

Это единственное для меня главное счастье во всей этой кромешной тьме непроглядной пурги, неприкрытых зонтами зимних вьюг, ног, навсегда перепрошитых жгучими пальцами ветра, колючих игл ледяных нитей моего отчаяния, неверия людей, вообще входящих порой глубоко корнями своих крохотинковых сердец, буквально впитавших всю боль утраты человеческого тепла, человечности, человеческих чувств, нечеловеческого сознания неспособного мыслить человека, животного существа

существо способное чувствовать лишь самого себя одним из нас троих выжившим после чудовищной болезни атлантов той эпохи, когда человек еще жив, могучий леденея сковывал своей ледяной корой наши самые маленькие

Для дообучения я выбрала тексты, где очень важна образность. Поэтому и получившийся результат "грешит" большим количеством оборотов, похожих на метафоры. Некоторые из них довольно странные, но я бы не сказала, что прямо абсурдные. Практически любую строчку можно истолковать и понять, какой смысл за ней может стоять. Помимо этого, получившиеся тексты соответствуют поданным реальным текстам тематически. 

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) GPT-1 была представлена в статье "Improving Language Understanding by Generative Pre-Training" (Alec Radford, Karthik Narasimhan, Tim Salimans, Ilya Sutskever) в 2018 году.

GPT-2 была представлена в статье "Language Models are Unsupervised Multitask Learners" (Alec Radford, Jeffrey Wu, Rewon Child, David Luan, Dario Amodei, Ilya Sutskever) в 2019 году.

GPT-3 была представлена в статье "Language Models are Few-Shot Learners" (Tom B. Brown, Benjamin Mann, Alec Radford, Jeffrey Wu, Rewon Child, David Luan, Dario Amodei, Ilya Sutskever) в 2020 году.

2) В целом, существуют наборы данных, которые являются достаточными для очень больших языковых моделей (the Common Crawl dataset2). Однако качество такого неотфильтрованного набора данных идеальным не назовешь. Поэтому создатели GPT-3 осуществили несколько действий для улучшения тренировочного датасета.  

(1) загрузили и отфильтровали версию CommonCrawl, основанную на сходстве с рядом высококачественных корпусов

(2) выполнили нечеткую дедупликацию на уровне документа, внутри наборов данных и между ними, чтобы предотвратить избыточность и сохранить целостность проверочного набора в качестве точной меры переобучения

(3) добавили известные высококачественные корпуса (WebText2, Books1, Books2, Wikipedia) в набор для обучения, чтобы расширить возможности CommonCrawl и увеличить его разнообразие

Во время обучения наборы данных не отбираются пропорционально их размеру: наборы данных, которые считаются более качественными, отбираются чаще.